In [ ]:
from matplotlib import pyplot
import numpy
from scipy.sparse import coo_matrix

import pyibm

%matplotlib inline

In [ ]:
pyibm.__version__

In [ ]:
config = dict(x=dict(start=-2.0, end=2.0, num_cells=20),
              y=dict(start=-2.0, end=2.0, num_cells=20))
grid = pyibm.GridBase(config=config)
grid

In [ ]:
gridc = pyibm.GridCellCentered(grid=grid)
gridx = pyibm.GridFaceX(grid=grid)
gridy = pyibm.GridFaceY(grid=grid)

In [ ]:
GHat = pyibm.assemble_GHat(gridc, gridx, gridy)
MHat = pyibm.assemble_MHat(gridx, gridy)
G = MHat @ GHat

DHat = pyibm.assemble_DHat(gridc, gridx, gridy)
R = pyibm.assemble_R(gridx, gridy)
RInv = pyibm.assemble_RInv(R)
D = DHat @ RInv

In [ ]:
Re = 100.0
LHat = 1 / Re * pyibm.assemble_LHat(gridx, gridy)
L = MHat @ LHat @ RInv

In [ ]:
dt = 0.01
alpha_implicit = 0.5
M = MHat @ RInv
A = 1 / dt * M - alpha_implicit * L

In [ ]:
MHatInv = pyibm.assemble_MHatInv(MHat)
MInv = R @ MHatInv
BN = pyibm.assemble_BN(gridx, gridy, dt=dt, N=3, L=L, MInv=MInv)

In [ ]:
DBNG = D @ BN @ G

In [ ]:
radius = 0.5
xc, yc = 0.0, 0.0
ds = gridx.x.get_widths()[0]
N = int(round(2 * numpy.pi * radius / ds))
epsilon = 0.0
theta = numpy.linspace(0.0, 2 * numpy.pi, num=N + 1)[:-1] + epsilon
x, y = xc + radius * numpy.cos(theta), yc + radius * numpy.sin(theta)
body = pyibm.Body(x, y, grid=gridc)
body

In [ ]:
Op = pyibm.assemble_delta(body, gridc, gridx, gridy,
                          kernel=pyibm.delta_roma_et_al_1999,
                          kernel_size=2)

In [ ]:
E = Op @ R @ MHat

S = pyibm.assemble_surfaces(body)
H = Op.T @ S

EBNH = E @ BN @ H

In [ ]:
p = pyibm.EulerianField('p', gridc, ic=0.0)
ux = pyibm.EulerianField('ux', gridx, ic=1.0)
uy = pyibm.EulerianField('uy', gridx, ic=0.0)